<a href="https://colab.research.google.com/github/luigiselmi/dl_tensorflow/blob/main/sequence_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep learning for text: sequence models
With the improvement using the bigram data we have seen that the order of words matters. Now we want to study more in depth how to exploit the order of words in texts to perform several tasks that are common when using textual data. Instead of creating N-grams we will let the model figure out how to use a text to address the task at hand using a sequence model. We start by assigning an integer index to each word in a vocabulary that will be mapped to a vector by computing the word embedding. One other approach was to map a word to a vector using the one-hot encoding technique. The problem with this technique is that the dimensionality of the vector space will be the same as the cardinality (size) of the vocabulary. The dimensionality of the embeddings can be much lower. These words embeddigs represent a base vectors that will be used to represent every text.

We replace the Tensorflow version available in Colab to install one that is closer to the version 2.6 used in the book by Chollet.

In [1]:
!pip install tensorflow==2.8

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 32.0 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully uninstalled tensorboard-data-server-0.7.2
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: goog

In [2]:
import tensorflow as tf
import keras
print('Tensorflow version: {}'.format(tf.__version__))
print('Keras version: {}'.format(keras.__version__))

Tensorflow version: 2.8.0
Keras version: 2.8.0


## The IMDB dataset
We download the same IMDB dataset used in the bag-of-words model

In [3]:
!wget -nv 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz' -P './data'

2024-10-30 08:43:11 URL:https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz [84125825/84125825] -> "./data/aclImdb_v1.tar.gz" [1]


In [4]:
!tar -xf data/aclImdb_v1.tar.gz -C data/

In [5]:
!rm -r data/aclImdb/train/unsup

The dataset is divided into a train and a test set. We take 20% of the reviews from the train set to create a validation set tha twill be used to monitor the performance of the model during the training.

In [6]:
import os, pathlib, shutil, random
base_dir = pathlib.Path("data/aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"
for category in ("neg", "pos"):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = int(0.2 * len(files))
    val_files = files[-num_val_samples:]
    for fname in val_files:
        shutil.move(train_dir / category / fname, val_dir / category / fname)

We instantiate three Keras Dataset to handle the datasets using an [utility function](https://keras.io/api/data_loading/text/) from keras.

In [7]:
from tensorflow import keras
batch_size = 32
train_ds = keras.utils.text_dataset_from_directory('data/aclImdb/train', batch_size=batch_size)
val_ds = keras.utils.text_dataset_from_directory('data/aclImdb/val', batch_size=batch_size)
test_ds = keras.utils.text_dataset_from_directory('data/aclImdb/test', batch_size=batch_size)
text_only_train_ds = train_ds.map(lambda x, y: x)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


We use only the first 600 words in the reviews and we set the vocabulary size to 20000 so each word will be mapped to an integer and a review will be represented as a vector of maximum 20000 integers.

In [8]:
from tensorflow.keras import layers
max_length = 600
max_tokens = 20000
text_vectorization = layers.TextVectorization(
  max_tokens=max_tokens,
  output_mode="int",
  output_sequence_length=max_length,
)

We prepare the dataset where each review example is an integer vector representation of its text and the label, 1 or 0.

In [9]:
text_vectorization.adapt(text_only_train_ds)
int_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
int_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
int_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)

## The sequence model
We can now create a first sequence model. One step is the computation of the words embeddings from the integer vector that we have used so far to represent a review. We set the dimensionality of the embeddings to 256. The number of parameters for the embeddings is the product of the integer vector dimensions, that is the size of the vocabulary, times the dimensionality of the embeddings. In other words, the shape of the embedding matrix is (vocabulary size, embedding dimensionality), in our case (20000, 256)

In [ ]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 256)         5120000   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               73984     
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 5,194,049
Trainable params: 5,194,049
Non-trainable params: 0
___________________________________________________

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru.keras",
                                    save_best_only=True)
]

In [ ]:
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)

Epoch 1/10
625/625 [==============================] - 459s 730ms/step - loss: 0.4689 - accuracy: 0.7896 - val_loss: 0.4226 - val_accuracy: 0.8120
Epoch 2/10
625/625 [==============================] - 453s 724ms/step - loss: 0.3011 - accuracy: 0.8901 - val_loss: 0.2993 - val_accuracy: 0.8838
Epoch 3/10
625/625 [==============================] - 458s 732ms/step - loss: 0.2538 - accuracy: 0.9135 - val_loss: 0.3076 - val_accuracy: 0.8842
Epoch 4/10
625/625 [==============================] - 451s 722ms/step - loss: 0.2053 - accuracy: 0.9298 - val_loss: 0.3093 - val_accuracy: 0.8810
Epoch 5/10
625/625 [==============================] - 447s 716ms/step - loss: 0.1714 - accuracy: 0.9427 - val_loss: 0.3492 - val_accuracy: 0.8848
Epoch 6/10
625/625 [==============================] - 454s 727ms/step - loss: 0.1501 - accuracy: 0.9517 - val_loss: 0.3239 - val_accuracy: 0.8854
Epoch 7/10
625/625 [==============================] - 446s 713ms/step - loss: 0.1258 - accuracy: 0.9603 - val_loss: 0.3454 -

The accuracy of the model is still somewhat lower than the bag-of-words model and the bigrams. One reason is that we use a subset of each review of 600 words. Another reason is that reviews shorter than 600 words are padded with zeros. We could reduce the second problem by setting the mask_zero argument of the Embedding layer to True so that the zeros will be masked out.

In [ ]:
model = keras.models.load_model("embeddings_bidir_gru.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

782/782 [==============================] - 101s 128ms/step - loss: 0.3371 - accuracy: 0.8709
Test acc: 0.871


## Pretrained word embeddings
We have seen that in our case learning the embeddings requires learning more than 5 millions parameters. We can avoid this step by using a pretrained embeddings that has a semantics, that is concepts and relationships between them, similar to that in our reviews.

In [10]:
!wget -nv 'http://nlp.stanford.edu/data/glove.6B.zip'  -P './data'

2024-10-30 08:47:55 URL:https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [862182613/862182613] -> "./data/glove.6B.zip" [1]


In [11]:
!unzip -q data/glove.6B.zip -d data/

We count the number of word embeddings in the pretrained dataset

In [12]:
import numpy as np
path_to_glove_file = "data/glove.6B.100d.txt"
embeddings_index = {}
with open(path_to_glove_file) as f:
  for line in f:
    word, coefs = line.split(maxsplit=1)
    coefs = np.fromstring(coefs, "f", sep=" ")
    embeddings_index[word] = coefs

print(f"Found {len(embeddings_index)} word vectors.")

Found 400000 word vectors.


In [13]:
embedding_dim = 100
vocabulary = text_vectorization.get_vocabulary()
word_index = dict(zip(vocabulary, range(len(vocabulary))))

embedding_matrix = np.zeros((max_tokens, embedding_dim))
for word, i in word_index.items():
  if i < max_tokens:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
      embedding_matrix[i] = embedding_vector

In [14]:
embedding_layer = layers.Embedding(
  max_tokens,
  embedding_dim,
  embeddings_initializer=keras.initializers.Constant(embedding_matrix),
  trainable=False,
  mask_zero=True,
)

We define a model with the pretrained embeddings so that the network does not need to learn the embedding parameters.

In [15]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = embedding_layer(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

callbacks = [
  keras.callbacks.ModelCheckpoint("glove_embeddings_sequence_model.keras",
  save_best_only=True)
]

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 100)         2000000   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               34048     
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 2,034,113
Trainable params: 34,113
Non-trainable params: 2,000,000
______________________________________________

In [16]:
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)

Epoch 1/10
625/625 [==============================] - 565s 887ms/step - loss: 0.5720 - accuracy: 0.6983 - val_loss: 0.4350 - val_accuracy: 0.8048
Epoch 2/10
625/625 [==============================] - 548s 876ms/step - loss: 0.4509 - accuracy: 0.7971 - val_loss: 0.4009 - val_accuracy: 0.8184
Epoch 3/10
625/625 [==============================] - 545s 872ms/step - loss: 0.4052 - accuracy: 0.8184 - val_loss: 0.3922 - val_accuracy: 0.8218
Epoch 4/10
625/625 [==============================] - 555s 888ms/step - loss: 0.3733 - accuracy: 0.8376 - val_loss: 0.3911 - val_accuracy: 0.8238
Epoch 5/10
625/625 [==============================] - 556s 890ms/step - loss: 0.3444 - accuracy: 0.8531 - val_loss: 0.3453 - val_accuracy: 0.8526
Epoch 6/10
625/625 [==============================] - 552s 882ms/step - loss: 0.3261 - accuracy: 0.8614 - val_loss: 0.3456 - val_accuracy: 0.8542
Epoch 7/10
625/625 [==============================] - 558s 893ms/step - loss: 0.3074 - accuracy: 0.8723 - val_loss: 0.3372 -

In [17]:
model = keras.models.load_model("glove_embeddings_sequence_model.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

782/782 [==============================] - 166s 207ms/step - loss: 0.2991 - accuracy: 0.8730
Test acc: 0.873
